In [ ]:
import pandas as pd
from pandas import Series,DataFrame

# numpy, matplotlib, seaborn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
%matplotlib inline

# machine learning
from sklearn.ensemble import RandomForestClassifier

In [ ]:
digits = pd.read_csv('train.csv')

In [ ]:
digits_test = pd.read_csv('test.csv')

In [ ]:
digits.head()
digits.tail()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier

alg1 = LogisticRegression(C = 2.5, max_iter = 100, tol = 0.0001, random_state = 5)
alg2 = RandomForestClassifier(n_estimators = 1000)
alg3 = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(1000, 1000, 1000, 1000), random_state=1)
alg4 = AdaBoostClassifier(n_estimators=500)
alg5 = SVC()
alg6 = neigh = KNeighborsClassifier(n_neighbors=10)
alg7 = GaussianNB()

eAlg = VotingClassifier(estimators=[('RF',alg2),('MLP',alg3),('Ada',alg4),('SVM',alg5),('NB',alg7)],
                        voting='hard',
                        weights=[1,1,1,1,1])

In [ ]:
#from keras.models import Sequential
#model = Sequential()

#from keras.layers import Dense, Activation
#model.add(Dense(output_dim=64, input_dim=100))
#model.add(Activation("relu"))
#model.add(Dense(output_dim=10))
#model.add(Activation("softmax"))

#from keras.optimizers import SGD
#model.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01, momentum=0.9, nesterov=True))
#alg = model

In [ ]:
predictors = []
for i in range(784):
    predictors.append('pixel'+ str(i))
#predictors

In [ ]:
digits.fillna(0, inplace = True)

In [ ]:
from sklearn.cross_validation import KFold
kf = KFold(digits.shape[0], n_folds=5, random_state=1)

predictions = []
for train, test in kf:
    train_predictors = digits[predictors].iloc[train,:]
    train_target = digits['label'].iloc[train]
    eAlg.fit(train_predictors, train_target)
    test_predictions = eAlg.predict(digits[predictors].iloc[test,:])
    predictions.append(test_predictions)

In [ ]:
np.array(predictions).shape

In [ ]:
import numpy as np
predictions = np.concatenate(predictions,axis=0)

from sklearn import metrics
accuracy = metrics.accuracy_score(digits["label"], predictions)
print(accuracy)

In [ ]:
digits_test.fillna(0, inplace = True)

# Train the algorithm using all the training data
eAlg.fit(digits[predictors], digits["label"])

# Make predictions using the test set.
predictions = eAlg.predict(digits_test[predictors])

# Create a new dataframe with only the columns Kaggle wants from the dataset.
submission = pd.DataFrame({
        "ImageId": digits_test.index + 1,
        "Label": predictions
    })

In [ ]:
submission.to_csv('digits.csv',index=None)

In [ ]:
import matplotlib.pyplot as plt
%pylab inline
digits.fillna(0, inplace = True)
train = digits.drop("label",1)

figure(figsize(10,10))
for digit_num in range(0,64):
    subplot(8,8,digit_num+1)
    grid_data = train.iloc[digit_num].as_matrix().reshape(28,28)  # reshape from 1d to 2d pixel array
    plt.imshow(grid_data, interpolation = "none", cmap = "bone_r")
    xticks([])
    yticks([])


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
pca = PCA(n_components=50)
pca.fit(train)
transform_train = pca.transform(train)
transform_test = pca.transform(digits_test)
target = digits['label']

clf = KNeighborsClassifier()
clf.fit(transform_train, target)
results=clf.predict(transform_test)

# prepare submit file

np.savetxt('results.csv', 
           np.c_[range(1,len(digits_test)+1),results], 
           delimiter=',', 
           header = 'ImageId,Label', 
           comments = '', 
           fmt='%d')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
import pandas as pd
from sklearn.preprocessing import StandardScaler

data= pd.read_csv('train.csv')
data.fillna(0, inplace = True)
target = data.pop('label').values
y_train = target[2000:]
y_valid = target[:2000]
X_train = data[2000:].values.astype('float32')
X_valid = data[:2000].values.astype('float32')

X_train = StandardScaler().fit(X_train).transform(X_train)
X_valid = StandardScaler().fit(X_valid).transform(X_valid)

X_train = X_train.reshape(-1, 28, 28, 1) # (40000, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1) # (2000, 28, 28, 1)


from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 10)
y_valid = np_utils.to_categorical(y_valid, 10)



model = Sequential()
model.add(Convolution2D(32, 3, 3, input_shape=(28,28,1), activation='relu',border_mode = 'valid'))
model.add(Convolution2D(32, 3, 3, border_mode='same'))
model.add(MaxPooling2D(pool_size=(2,2), dim_ordering='th'))
model.add(Dropout(0.3))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

num_epochs = 6
model.compile(optimizer='adadelta', loss='categorical_crossentropy', metrics=['accuracy'])

hist = model.fit(X_train,y_train, batch_size=128, nb_epoch=6, 
                validation_data=(X_valid,y_valid))
scores = model.evaluate(X_valid, y_valid, verbose=0)

import matplotlib.pyplot as plt 

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss Rate')
plt.ylabel('Loss')
plt.xlabel('Training interations')
plt.legend(['Training', 'Testing'], loc='upper left')
plt.show()
#plt.savefig('MNIST_loss_plot1.png')

plt.plot(hist.history['val_acc'])
plt.title('Accuracy Rate')
plt.ylabel('Accuracy %')
plt.xlabel('Training iterations')
plt.legend(['Testing'], loc='upper left')
plt.show()
#plt.savefig('MNIST_acc_plot1.png')

# load test data
test_data = pd.read_csv('test.csv').values.astype('float32')
test_data = StandardScaler().fit(test_data).transform(test_data)
test_data = test_data.reshape(-1,28,28,1)

test_submission = model.predict_classes(test_data, verbose=2)

In [21]:
pd.DataFrame({"ImageId": list(range(1,len(test_data)+1)), 
              "Label": test_submission}).to_csv('CNN.csv', index=False,header=True)

In [24]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
